#### 7_2_convolutions_for_images.ipynb
Image data is represented as a two-dimensional grid of pixels, be the image monochromatic or in color. 

Accordingly each pixel corresponds to one or multiple numerical values respectively. 

So far we have ignored this rich structure and treated images as vectors of numbers by flattening them, 

irrespective of the spatial relation between pixels. 

This deeply unsatisfying approach was necessary in order to feed the resulting one-dimensional vectors through a fully connected MLP.

#### 空间不变性
我们可以从儿童游戏“寻找沃尔多”（Where's Waldo）中汲取一些启示（这个游戏本身已经激发了许多现实生活中的模仿，如图 7.1.1 所示）。游戏包含许多充满活动的混乱场景。在每个场景中，沃尔多都会出现在某个地方，通常潜伏在一些不太可能的地方。读者的目标是找到他。尽管沃尔多有着特征性的服装，找到他却常常出奇地困难，因为场景中存在大量的干扰。然而，沃尔多的样子并不取决于他位于画面的哪个位置。我们可以用一个沃尔多探测器在图像上扫描，为每个区域分配一个分数，表示该区域包含沃尔多的可能性。事实上，许多目标检测和分割算法都是基于这种方法(Long et al., 2015)。卷积神经网络（CNNs）系统化了空间不变性这一思想，并利用它来学习具有较少参数的有用表示。

在实践中，卷积神经网络通过卷积层实现图像数据的空间不变性。卷积层使用卷积核（也称滤波器）在图像数据上滑动，从而提取局部特征。在处理图像数据时，CNN 不仅减少了参数的数量，还能更有效地提取图像特征。这种用较少参数获得更好特征表示的能力，使得卷积神经网络在图像识别、目标检测和语义分割等任务上取得了卓越的性能。

为了有效地处理空间变换，卷积神经网络还使用了汇聚层（Pooling）来降低输入数据的空间维度，减少计算复杂度。这有助于在保持不变性的同时，对特征图进行逐层抽象，使得网络能够捕捉更复杂和高级的特征表示。

总而言之，通过利用空间不变性以及设计灵活的多层结构，卷积神经网络可以高效地分析图像，检测或定位正在寻找的目标，就像一个寻找沃尔多的侦察员一样。这些技术已经在计算机视觉和深度学习领域取得了重大突破，并帮助实现了更有效和可靠的图像识别算法。

#### 基于这种诉求，卷积神经网络应该做到
We can now make these intuitions more concrete by enumerating a few desiderata to guide our design of a neural network architecture suitable for computer vision:

1. In the earliest layers, our network should respond similarly to the same patch, regardless of where it appears in the image. This principle is called translation invariance (or translation equivariance).

2. The earliest layers of the network should focus on local regions, without regard for the contents of the image in distant regions. This is the locality principle. Eventually, these local representations can be aggregated to make predictions at the whole image level.

3. As we proceed, deeper layers should be able to capture longer-range features of the image, in a way similar to higher level vision in nature.

#### 结论
在本节中，我们从基本原理推导出卷积神经网络的结构。虽然目前尚不清楚这是否是导致 CNN 发明的路径，但至少在较低层次上，知道它是应用合理原则处理图像处理和计算机视觉算法的正确选择让人十分满意。尤其是，图像中的平移不变性意味着图像的所有区域都将以相同的方式处理。局部性意味着仅使用像素的小范围邻域来计算相应的隐藏表示。最早提及 CNN 的参考文献之一是 Neocognitron（Fukushima，1982）。

我们在推理过程中遇到的第二个原则是如何在不限制其表达能力的情况下（至少在模型满足某些假设时）减少函数类中参数的数量。结果，我们看到了复杂性的显著降低，将计算和统计上不可行的问题转化为可解问题。

添加通道使我们能够在局部性和平移不变性对卷积核施加限制的情况下，恢复一些丢失的复杂性。注意，除了红、绿、蓝之外，添加其他通道是很自然的。许多卫星图像，尤其是农业和气象领域的图像，具有数十到数百个通道，生成超光谱图像。它们会记录各种波长的数据。接下来，我们将了解如何有效地使用卷积来操作图像的维度，如何从基于位置的表示转向基于通道的表示，以及如何有效地处理大量类别。

#### 问答部分
1. 当卷积核大小 A=0 时，卷积核会为每组通道单独实现一个多层感知机（MLP）。具体来说，在这种情况下，输入的通道仅与相应的输出通道相互作用，而不会与同一组的其他通道产生局部交互作用。这种结构导致了 Network in Network（NiN）架构（Lin et al., 2013），其中在卷积层内执行单独的 MLP 来使局部信息融合。

2. 音频数据通常表示为一维序列。

   2.1 在处理音频时，您可能会想要实现局部性和平移不变性，以便识别音频中的特定模式或音高，以及它们在音频信号中的出现位置。

   2.2 音频卷积操作的推导类似于图像卷积操作，只是在一维序列数据中进行计算，而非二维数据。音频卷积输入一个信号 f(t) 和卷积核 g(t)，输出一个新信号 h(t)，其计算公式为：h(t) = ∫ [f(τ) * g(t - τ)] dτ。

   2.3 可以使用与计算机视觉相同的工具处理音频。提示：将音频信号通过其语谱图（spectrogram）转换为二维表示。在语谱图中，横坐标表示时间，纵坐标表示频率。这使得我们可以将二维卷积操作应用于音频数据，类似于图像处理。

3. 平移不变性可能并不总是一个好主意，因为它会导致某些任务中关键的空间信息丢失。例如，在物体定位任务中，我们需要精确地确定物体在图像中的位置，同时识别出物体。在这种情况下，使用平移不变性会削弱模型的定位能力。

4. 卷积层也可以应用于文本数据。由于文本数据具有自然的一维序列结构（单词或字符），卷积层可以用于捕捉局部依赖和语义信息。然而，在处理语言时可能会遇到一些问题，如处理不同长度的文本、词汇歧义、处理歧义等。解决这个问题的一种方法是使用词嵌入来表示文本，并在输入卷积神经网络之前进行预处理。

5. 当物体位于图像边界时，由于卷积核可能超出图像范围，卷积操作可能会丢失部分信息。为解决这个问题，通常引入填充（Padding）策略，在图像边缘添加额外的像素值（例如，填充值为0）以应对边界效应。

6. 证明卷积是对称的，即 f*g = g*f。

   假设 f(t) 和 g(t) 是两个信号函数，卷积操作定义为 h(t) = ∫ [f(τ) * g(t - τ)] dτ。我们需要证明 f*g = g*f。通过改变积分变量 u = t - τ 可得：

   f * g (t) = ∫ [f(τ) * g(t - τ)] dτ
   = ∫ [f(t - u) * g(u)] du (变换积分变量 τ = t - u)
   = ∫ [g(u) * f(t - u)] du
   = g * f (t)

   由此可证，卷积是对称的，即 f*g = g*f。

#### why 卷积可以降低参数复杂度（计算复杂度）

首先，我们来分析一下为什么参数数量会增长到 10^12。假设我们处理一张 1000×1000 像素的图像（1 百万像素），并将其映射到一个具有相同尺寸的 1000×1000 的隐藏表示（即每个像素对应一个隐藏层神经元）。在这个隐藏表示中，我们将考虑输入图像的每个像素与隐藏层的每个神经元的连接权重。

使用传统的全连接神经网络，每个输入像素都要与隐藏层的每个神经元相连。换句话说，每个像素在输入图像中的位置都会连接到隐藏层中的 1000×1000 个神经元。这将导致：

1000（输入像素宽度）× 1000（输入像素高度）× 1000（隐藏表示宽度）× 1000（隐藏表示高度）= 10^12 个参数

这个数量的参数是非常巨大的，远远超过了现代计算机所能处理的数目。这种指数级增长的参数量会导致计算效率低下，极易导致过拟合，而且显存资源耗尽。为了解决这个问题，卷积神经网络（CNN）应用了局部连接和权值共享的技巧，这显著降低了网络的参数数量。因此，在处理具有大量参数的图像时，卷积神经网络相对全连接神经网络具有较高的计算效率和性能。


卷积神经网络（CNN）通过卷积层显著降低参数数量。让我们用一个例子来说明为什么卷积可以将参数复杂度降低到 4 * 10^6。

假设我们有一个 1000×1000 的输入图像，是一个 1 百万像素的照片。然后，我们应用一个卷积层，使用的是 3×3 的卷积核（或过滤器）和 64 个输出通道。实际上，我们只需要学习每个卷积核（滤波器）的参数，而不是像全连接层那样分别学习每个输入-输出像素对的连接权重。

对于这个例子中的卷积层，我们需要学习的总参数为：

卷积核宽度 × 卷积核高度 × 输入通道数 × 输出通道数

假设输入图像是彩色的，即具有 3 个通道（RGB），那么计算公式如下：

3（输入通道数）× 3（卷积核高度）× 3（卷积核宽度）× 64（输出通道数）= 1728 个参数

这只是卷积操作完成之后的参数数量。接下来，我们得到了一个具有 1000×1000 的隐藏表示和 64 个通道的输出特征图。我们可以使用一个全连接层或另一个卷积层将其映射到最终输出分数。假设我们的任务是一个 1000 类分类问题，最后一层需要学习的参数数量为：

1000（类别数）× 64（上一层输出通道数）= 64,000 个参数

因此，整个卷积神经网络需要的参数总数为：1728（卷积层参数）+ 64,000（输出层参数）= 65,728 个参数。显然，相对于 10^12 个参数，这个数量明显降低，使 CNN 变得更高效和可行。

请注意，上述例子仅用于说明 CNN 如何减少参数数量。实际上，CNN 可能包含多个卷积层和其他结构。但总的来说，卷积神经网络在降低参数数量和处理具有大量参数的输入图像方面要优于全连接神经网络。这使得卷积神经网络在处理计算机视觉任务和大规模图像处理等应用场景中具有优势。